Simple model of lactose pathway in E Coli

This is an attempt to build and compare a kinetic metabolic model of the lactose pathway in E Coli. There are 3 steps in this pathway as illustrated by Dykhuizen and Dean (1990). The first is diffusion of lactose through porins in the cell wall into the periplasm. The second is active transport of lactose across the cell membrane into the cytoplasm. The third is hydrolysis of the lactose into glucose and galactose by Beta-galactosidase. This model will have 3 compartments and 3 different types of rate laws, and should be a good test illustration for developing these types of models. In this version the second step is represented by passive diffusive transport.

In [24]:
import tellurium as te
import numpy as np
import matplotlib.pyplot as plt

In [35]:
    fmol_mol = 1.0e15 ; # femtomols per mol
    sec_day = 60.*60.*24 ; # seconds per day
    ml_liter = 1.0e3 ; # ml per liter
    
    cell_radius = 1.0e-5 ; # dm, for cell radius of 1 micron
    periplasm_width = 10.0e-8 ; # dm, for periplasm width of 10 nanometers
    
    cell_density = 1.0e9 ; # free parameter cells ml^-1
    culture_volume = 1.0e1 ; # ml
    cell_numbers = cell_density * culture_volume
    
    area_cell = 4.0 * np.pi * (cell_radius)**2
    volume_cell = 4.0 / 3.0 * np.pi * (cell_radius)**3
    area_cytoplasm = 4.0 * np.pi * (cell_radius - periplasm_width)**2
    volume_cytoplasm = 4.0 / 3.0 * np.pi * (cell_radius - periplasm_width)**3
    volume_periplasm = volume_cell - volume_cytoplasm
  
    permeability_23 = 1.0e-8 ; # dm s^-1, assuming 1.0e-7 cm s^-1 from MBotC   
    area_23 = area_cell
    
    volume_34 = volume_cell
    kcat_34 = 6.42e2 ; #s^-1, from Juers et al (2012) 
    Etot_34 = 1.0e-6 ; # M, free parameter
    Km_34 = 5.5e-3 ; # M, from BRENDA  
    
    S2 = 1.0e-2 #np.linspace(1.0e-3, 100.e-3, 100) ; # M, free parameter
    
    ayy = 1.0e0
    bee = (Km_34 - S2 + kcat_34 * Etot_34 * volume_34 / (permeability_23 * area_23))
    cee = -1.0e0 * Km_34 * S2
    
    S3_ss = (-1.0e0 * bee + np.sqrt(bee**2 - 4.0e0 * ayy * cee)) / (2.0e0 * ayy)
    
    Jss_23 = (permeability_23 * area_23) * (S2 - S3_ss)
    Jss_34 = kcat_34 * Etot_34 * volume_34 * S3_ss / (Km_34 + S3_ss)
    
    #logS2 = boz.log10(S2)
    #logJss_23 = boz.log10(Jss_23)
    #te.plot(logS2, logJss_23) 
    #te.plot(S2, Jss_23, logx = True, logy = False)
    
    k_ss = Jss_23 / (S2 * culture_volume / ml_liter) * cell_numbers 
    half_life_ss = -1.0e0 * np.log(0.5) / k_ss
    
    #te.plot(S2, half_life_ss)
    print('steady_state_rate_per_cell',Jss_23)
    print('steady_state_rate_constant_per_culture',k_ss)
    print('half_life (days)',half_life_ss / sec_day )
    print('cell_numbers', cell_numbers)
    print('mols of initial lactose',(S2 * culture_volume / ml_liter))
    
    #print('V cell', volume_cell)
    #print('SA cell', area_cell)
    #print('S3_ss', S3_ss, 'M', S3_ss * 1.0e6, 'microM')
    #print('J_ss', Jss_23, 'mol cell^-1 s^-1', Jss_23 * fmol_mol * sec_day, 'Jørgs')
    #print('k_ss', k_ss, 'sec^-1', k_ss * sec_day, 'days^-1')
    #print('half_life_ss', half_life_ss, 'sec', half_life_ss / sec_day, 'days' )

steady_state_rate_per_cell 1.2236878275758288e-19
steady_state_rate_constant_per_culture 1.2236878275758289e-05
half_life (secs) 0.655603220956226
cell_numbers 10000000000.0
mols of initial lactose 0.0001


In [69]:
model LacPath
    
    compartment C1, C2, C3;
    
    C1 = culture_volume / ml_liter; # culture volume in liters
    C2 = cell_numbers * volume_periplasm; # total periplasmic volume in liters
    C3 = cell_numbers * volume_cytoplasm; # total cytoplasmic volume in liters
    
    species S1, S2, S3, S4;
    
    S1 in C1;
    S2 in C2;
    S3 in C3;
    S4 in C3;
    
    S1 = 10.0e-3 ; # M, free parameter
    S2 = 0.0
    S3 = 0.0
    S4 = 0.0
    
    fmol_mol = 1.0e15 ; # femtomols per mol
    sec_day = 60.*60.*24 ; # seconds per day
    ml_liter = 1.0e3 ; # ml per liter
    
    cell_radius = 1.0e-5 ; # dm, for cell radius of 1 micron
    periplasm_width = 10.0e-8 ; # dm, for periplasm width of 10 nanometers
    cell_density = 1.0e9 ; # free parameter cells ml^-1
    culture_volume = 1.0e0 ; # ml
    cell_numbers = cell_density * culture_volume
   
    area_cell = 4.0 * pi * (cell_radius)^2
    volume_cell = 4.0 / 3.0 * pi * (cell_radius)^3
    area_cytoplasm = 4.0 * pi * (cell_radius - periplasm_width)^2
    volume_cytoplasm = 4.0 / 3.0 * pi * (cell_radius - periplasm_width)^3
    volume_periplasm = volume_cell - volume_cytoplasm
  
    permeability_12 = 1.0e-8 ; # dm s^-1, assuming 1.0e-7 cm s^-1 from MBotC 
    area_12 = area_cell
  
    permeability_23 = 1.0e-8 ; # dm s^-1, assuming 1.0e-7 cm s^-1 from MBotC   
    area_23 = area_cytoplasm
    
    volume_34 = volume_cytoplasm
    kcat_34 = 6.42e2 ; #s^-1, from Juers et al (2012) 
    Etot_34 = 1.0e-6 ; # M, free parameter
    Km_34 = 5.5e-3 ; # M, from BRENDA
  
    J12: S1 -> S2; cell_numbers * permeability_12 * area_12 * (S1 - S2);
    J23: S2 -> S3; cell_numbers * permeability_23 * area_23 * (S2 - S3);
    J34: S3 -> S4; cell_numbers * volume_34 * kcat_34 * Etot_34 * S3 / (Km_34 + S3);
   
    k_12 = J12 / (S1 * C1) 
    half_life_12 = -1.0e0 * log(0.5) / k_12 / sec_day
    
end


Success: Model can be accessed via variable LacPath


In [70]:
LacPath.reset()
LacPath.simulate(start = 0., end = 1e5, steps = 100, selections=['time',"S1", "S2", "S3", "S4"]) 

LacPath.plot()

selection_reactionids = LacPath.getReactionIds()
selection_reactionrates = LacPath.getReactionRates()

print(selection_reactionids)
print(selection_reactionrates)


['J12', 'J23', 'J34']
[3.32251881e-11 3.32273029e-11 3.32290884e-11]
